## Topic Modeling using Sigular Value Decomposition
  * Singular Value Decomposion is a Linear Algebraic concept used in may areas such as machine learning (principal component analysis, Latent Semantic Analysis, Recommender Systems and word embedding), data mining and bioinformatics.
  * The technique decomposes given matrix into there matrices, let's look at programmer's perspective of the technique.

In [1]:
import numpy as np
import pandas as pd

#### Let's take a small dataset and create Document-Term matrix.
  * Note: Need to build Term-Document matrix For Query Retrieval in IR task. 

In [2]:
documents = ["I enjoy Machine Learning", "I like NLP", "I like deep learning"]

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(1, 1))
X = cv.fit_transform(documents).toarray()

#### X is the Document-Term Matrix, "DOCUMENTS" - are the "rows";  "TOKENS" - are the "columns" of the matrix

In [4]:
X

array([[0, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1],
       [1, 0, 1, 1, 0, 0]], dtype=int64)

In [5]:
cv.get_feature_names()

['deep', 'enjoy', 'learning', 'like', 'machine', 'nlp']

In [6]:
cv.vocabulary_

{'enjoy': 1, 'machine': 4, 'learning': 2, 'like': 3, 'nlp': 5, 'deep': 0}

##### Notation 
  * "*" - for Matrix multiplication
  * V.T - to denote Transpose of matrix V
  
#### Now SVD will decompose rectangular matrix X into there matrices. Namely U, S and V.T (V transpose)
  * U - the left Eigen vector (orthogonal) matrix, the vectors extracts "Document-to-Document similarity". 
  * V - the right Eigen vector (orthogonal) matrix, the vectors extract "Strength of Two Different Words Occurring in one Document" (may or may-not be co-occurrence)
  * S - the Eigen Values (diagonal) matrix, explains variance ratio.
#### Before we understand how U, V, S extract relation strengh or explain variance ratio. Let's understand on how to decompose X.
  * To decomposing any matrix, we should use Eigen Vectors, Eigen Values decomposition.
  * A matrix is **eligible** to apply Eigen decomposition only when it is a **square and symmetric matrix*.
  * We can get a SQUARE, SYMMETRIC matrix by multiplying "X.T with X" or "X with X.T", we can decompose these (np.dot(X.T,X) and np.dot(X,X.T) and get U, S, V matrixes.
#### Why to multiply "X.T with X" or "X with X.T"?, Why to decompose "X.T with X" or "X with X.T"?
  * **Suppose that X is decomposed as U S V.T**
    * **i.e., X = U * S * V.T**
    
  * Now let's see what is X.T multiplied by X
    * X.T * X = (U * S * V.T).T * (U * S * V.T)
    * &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        = (V * S.T * U.T) * (U * S * V)
    * &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        = V * S.T * (U.T * U) * S * V)
    * &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        = V * S.T * (I) * S * V    # as U is an orthogonal matrix U.T * U = I
    * &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        = V * (S.T * S) * V
    * &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        = V * S^2 * V      # as S is diagnoal matrix S.T * S is S-square    
  * We got V * S^2 * V, when we multiply X.T with X, where X is in decomposed form (U * S * V.T), **Hence, when we decompose X.T * X, we get values of matrix V and S^2**
  
  * Now let's see what is X multiplied by X.T
    * X * X.T = (U * S * V.T) * (U * S * V.T).T
    * &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        = (U * S * V) * (V.T * S.T * U)
    * &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        = (U * S * (V * V.T) * S.T * U)
    * &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        = U * S * (I) * S.T * V    # as U is an orthogonal matrix U.T * U = I
    * &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        = U * (S.T * S) * U
    * &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        = U * S^2 * U      # as S is diagnoal matrix S * S.T is S-square    
  * We got U * S^2 * U, when we multiply X.T with X, where X is in decomposed form (U * S * V.T), **Hence, when we decompose X * X.T, we get values of matrix U and S^2**
  
  * **Bottom line: To find U, S and V, we need to find Eigen Decoposition of X.T * X and X * X.T**

#### X.T * X is word similarity matrix, the (i, j) position in X.T * X will quantify the overlap of Word-i and Word-j. Number of Documents in which both words i and j occur. (This inference is based out of Document-Term matrix, if it is Tf-IDF, then the inference would be different)

In [7]:
cv.get_feature_names()

['deep', 'enjoy', 'learning', 'like', 'machine', 'nlp']

In [8]:
cv.vocabulary_

{'enjoy': 1, 'machine': 4, 'learning': 2, 'like': 3, 'nlp': 5, 'deep': 0}

In [9]:
np.dot(X.T, X)

array([[1, 0, 1, 1, 0, 0],
       [0, 1, 1, 0, 1, 0],
       [1, 1, 2, 1, 1, 0],
       [1, 0, 1, 2, 0, 1],
       [0, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1]], dtype=int64)

#### X * X.T is document similarity matrix, the (i, j) position in X * X.T will quantify the similarity of Document-i and Document-j. This analogy suits best with Tf-IDF scores.

In [10]:
np.dot(X, X.T)

array([[3, 0, 1],
       [0, 2, 1],
       [1, 1, 3]], dtype=int64)

####  The rows in sigular vector matrix V.T are the topics and values in the row denote strenght of words in the topic.

In [11]:
U, s, Vh = np.linalg.svd(X) 

In [12]:
U.shape

(3, 3)

In [13]:
Vh.shape

(6, 6)

In [14]:
print(Vh)

[[ 0.35761308  0.28678342  0.6443965   0.51676587  0.28678342  0.15915279]
 [ 0.20519296 -0.4610644  -0.25587144  0.5749379  -0.4610644   0.36974494]
 [-0.53995111  0.29965026 -0.24030085  0.13335691  0.29965026  0.67330802]
 [-0.6580513   0.1075659   0.19856672  0.45948457 -0.30613262 -0.45948457]
 [-0.20636755 -0.71288399  0.07368819  0.13267936  0.6391958  -0.13267936]
 [-0.250684   -0.30920965  0.64550737 -0.39482338 -0.33629772  0.39482338]]


In [17]:
print(s)

[2.06082013 1.59842364 1.09456031]


In [15]:
for i, comp in enumerate(Vh):
    print('topic-',i, '; scores for features - ', comp)
    z = zip(cv.get_feature_names(), comp)
    for a, b in z:
        print('\"featuer\": ',a, '     \"score\": ', b)
    print()

topic- 0 ; scores for features -  [0.35761308 0.28678342 0.6443965  0.51676587 0.28678342 0.15915279]
"featuer":  deep      "score":  0.357613077803957
"featuer":  enjoy      "score":  0.28678342190830464
"featuer":  learning      "score":  0.6443964997122618
"featuer":  like      "score":  0.5167658699398142
"featuer":  machine      "score":  0.28678342190830475
"featuer":  nlp      "score":  0.1591527921358573

topic- 1 ; scores for features -  [ 0.20519296 -0.4610644  -0.25587144  0.5749379  -0.4610644   0.36974494]
"featuer":  deep      "score":  0.2051929597703755
"featuer":  enjoy      "score":  -0.461064395430453
"featuer":  learning      "score":  -0.2558714356600775
"featuer":  like      "score":  0.5749378971020997
"featuer":  machine      "score":  -0.46106439543045297
"featuer":  nlp      "score":  0.36974493733172425

topic- 2 ; scores for features -  [-0.53995111  0.29965026 -0.24030085  0.13335691  0.29965026  0.67330802]
"featuer":  deep      "score":  -0.53995110647420

In [16]:
terms = cv.get_feature_names()
for i, component in enumerate(Vh):
    terms_components = zip(terms, component)
    sorted_terms = sorted(terms_components, key=lambda x:x[1], reverse=True)[:3] # take features for topic
    print("topic : ", i)
    for term_socres in sorted_terms:
        print(10*" ", term_socres[0])
    print(50*'*')

topic :  0
           learning
           like
           deep
**************************************************
topic :  1
           like
           nlp
           deep
**************************************************
topic :  2
           nlp
           enjoy
           machine
**************************************************
topic :  3
           like
           learning
           enjoy
**************************************************
topic :  4
           machine
           like
           learning
**************************************************
topic :  5
           learning
           nlp
           deep
**************************************************


### The above result may not make sense as the dataset is too small. If we take decent amount data, the results will be promising

#### Cons:
  * SVD is a linear decomposition, hence cannot extract non-linear patterns
  * SVD computation is very expensive, cannot handle huge volumens of data (Randomized SVD is one of the solutions)